In [32]:
import pandas as pd
import numpy as np
import sklearn
import pandas as pd
import seaborn as sns
import altair as alt
from sklearn.model_selection import train_test_split

In [2]:
penguin_data_path = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-07-28/penguins.csv'

In [3]:
pg_data = pd.read_csv(penguin_data_path)

In [4]:
pg_data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [5]:
# How many years were these studies over?
pg_data['year'].unique()

array([2007, 2008, 2009])

In [11]:
# How many species are there throughout the years?
pg_data['species'].unique()

array(['Adelie', 'Gentoo', 'Chinstrap'], dtype=object)

In [12]:
# Were the entry counts equal across years?  
pg_data.groupby('year').count().sort_values(by = 'year')

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
year,,,,,,,
2007,110,110,109,109,109,109,103
2008,114,114,114,114,114,114,113
2009,120,120,119,119,119,119,117


In [14]:
 alt.Chart(pg_data).mark_circle(size = 69).encode(
     x='bill_length_mm',
     y='body_mass_g', 
     color = 'species',
     tooltip=['species', 'bill_length_mm', 'body_mass_g']
 ).interactive()

alt.Chart(...)

In [15]:
 alt.Chart(pg_data).mark_circle(size = 69).encode(
     x='flipper_length_mm',
     y='body_mass_g', 
     color = 'species',
     tooltip=['species', 'flipper_length_mm', 'body_mass_g']
 ).interactive()

alt.Chart(...)

In [16]:
alt.Chart(pg_data).mark_circle(size = 69).encode(
     x='bill_depth_mm',
     y='body_mass_g', 
     color = 'species',
     tooltip=['species', 'bill_depth_mm', 'body_mass_g']
 ).interactive()

alt.Chart(...)

In [23]:
pg_data.groupby(['year', 'sex'])['species'].count().reset_index()

,year,sex,species
0,2007,female,51
1,2007,male,52
2,2008,female,56
3,2008,male,57
4,2009,female,58
5,2009,male,59


In [24]:
pg_data.groupby(['year', 'sex'])['species'].count().reset_index().sort_values('species', ascending = False)

,year,sex,species
5,2009,male,59
4,2009,female,58
3,2008,male,57
2,2008,female,56
1,2007,male,52
0,2007,female,51


In [29]:
pg_data.groupby(['year', 'sex', 'species'])['island'].count().reset_index().sort_values(['sex', 'species', 'year'], ascending = False)

,year,sex,species,island
17,2009,male,Gentoo,21
11,2008,male,Gentoo,23
5,2007,male,Gentoo,17
16,2009,male,Chinstrap,12
10,2008,male,Chinstrap,9
4,2007,male,Chinstrap,13
15,2009,male,Adelie,26
9,2008,male,Adelie,25
3,2007,male,Adelie,22
14,2009,female,Gentoo,20


In [ ]:
# Preprocess
## dump all na's throughout the df
## dummy encode island
## dummy encode species
## dummy encode sex


In [45]:
pg_data[pg_data.sex.isna() == False].isna().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
year                 0
dtype: int64

In [46]:
# remove na's
pg_data = pg_data[pg_data.sex.isna() == False]

# split x and y
X = pg_data[[x for x in pg_data.columns.values  if x != 'sex']]
y = pg_data['sex']

In [47]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 8675309)

In [141]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, accuracy_score
from sklearn.utils import resample
from tqdm import tqdm

In [77]:
log_reg = LogisticRegression(solver = 'lbfgs',
                            random_state = 8675309,
                             max_iter = 500)

In [78]:
column_trans = make_column_transformer((OneHotEncoder(), 
                        ['island', 'species']),
                        remainder = 'passthrough')

In [79]:
preproc_pipeline = make_pipeline(column_trans, 
                                 log_reg)

In [80]:
cross_val_score(preproc_pipeline, X_train, y_train, cv = 10, scoring = "accuracy").mean()

0.9361823361823362

In [82]:
preproc_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['island', 'species'])],
                                   verbose=False)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                       

In [107]:
test_y_score = preproc_pipeline.predict_proba(X_test)[:, 1]
test_y_label = preproc_pipeline.predict(X_test)

In [106]:
roc_auc = roc_auc_score(y_true = y_test.values,
              y_score = test_y_score)

In [112]:
conf_matrix = confusion_matrix(y_test, test_y_label)
conf_matrix

array([[34,  7],
       [ 3, 23]])

In [122]:
fpr, tpr, thresholds = roc_curve(y_test, test_y_score, pos_label = "male")
roc_frame = pd.DataFrame(data = {'fpr': fpr,
                     'tpr':tpr,
                     'threshold': thresholds})

In [125]:
alt.Chart(roc_frame).mark_line().encode(
    x='fpr',
    y='tpr'
).interactive()

alt.Chart(...)

In [140]:
accuracy_score(y_true = y_test, y_pred = test_y_label)

0.8507462686567164

In [138]:
help(sklearn.metrics.accuracy_score)

Help on function accuracy_score in module sklearn.metrics._classification:

accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)
    Accuracy classification score.
    
    In multilabel classification, this function computes subset accuracy:
    the set of labels predicted for a sample must *exactly* match the
    corresponding set of labels in y_true.
    
    Read more in the :ref:`User Guide <accuracy_score>`.
    
    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) labels.
    
    y_pred : 1d array-like, or label indicator array / sparse matrix
        Predicted labels, as returned by a classifier.
    
    normalize : bool, optional (default=True)
        If ``False``, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    
    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.
    

In [128]:
# creating bootstrapped datasets
boot = resample(pg_data, replace=True, random_state=8675309)

In [218]:
# bootstraping

summary_frame = pd.DataFrame()
roc_frame = pd.DataFrame()
for i in tqdm(range(25)):
  # resample the data frame
  boot = resample(pg_data, replace=True, random_state=i)
  X = boot[[x for x in boot.columns.values  if x != 'sex']]
  y = boot['sex']
  X_train, X_test, y_train, y_test = train_test_split(X, y)
  preproc_pipeline.fit(X_train, y_train)
  test_y_score = preproc_pipeline.predict_proba(X_test)[:, 1]
  test_y_label = preproc_pipeline.predict(X_test)
  roc_auc = roc_auc_score(y_true = y_test.values,
              y_score = test_y_score)
  acc = accuracy_score(y_true = y_test, y_pred = test_y_label)
  fpr, tpr, thresholds = roc_curve(y_test, test_y_score, pos_label = "male")
  # add on the roc and acc metrics
  summary_row = pd.DataFrame([[i, roc_auc, acc]], columns=['bootstrap', 'roc_auc', 'accuracy'])
  summary_frame = summary_frame.append(summary_row)

  # add on fpr, tpr, thresholds
  roc_rows = pd.DataFrame({'bootstrap': np.repeat(str(i), len(fpr)), 
                           'fpr': fpr,
                           'tpr': tpr,
                           'threshold': thresholds})
  roc_frame = roc_frame.append(roc_rows)





  0%|          | 0/25 [00:00<?, ?it/s]


  8%|▊         | 2/25 [00:00<00:01, 17.43it/s]


 16%|█▌        | 4/25 [00:00<00:01, 17.27it/s]


 24%|██▍       | 6/25 [00:00<00:01, 16.92it/s]


 32%|███▏      | 8/25 [00:00<00:00, 17.35it/s]


 44%|████▍     | 11/25 [00:00<00:00, 18.48it/s]


 52%|█████▏    | 13/25 [00:00<00:00, 18.01it/s]


 60%|██████    | 15/25 [00:00<00:00, 17.55it/s]


 68%|██████▊   | 17/25 [00:00<00:00, 18.05it/s]


 80%|████████  | 20/25 [00:01<00:00, 18.83it/s]


 88%|████████▊ | 22/25 [00:01<00:00, 18.06it/s]


100%|██████████| 25/25 [00:01<00:00, 17.65it/s]


In [153]:
summary_frame[['roc_auc', 'accuracy']].mean()

roc_auc     0.974952
accuracy    0.910952
dtype: float64

In [183]:
summary_melt = summary_frame.melt(id_vars = 'bootstrap', var_name = 'summary_metric') \

summary_mean = summary_melt[['summary_metric', 'value']] \
  .groupby('summary_metric', as_index = False) \
  .mean() \
  .rename(columns = {'value': 'mean'})

summary_melt = summary_melt.merge(summary_mean, 
                                  how = 'inner',
                                  on = ['summary_metric'])

In [196]:
summary_melt.columns.values

array(['bootstrap', 'summary_metric', 'value', 'mean'], dtype=object)

In [204]:
chart_one = alt.Chart().mar k_bar().encode(
    alt.X("value:Q", bin=True),
    y='count()',
    color = 'summary_metric',
    strokeDash = 'mean') #,
    # column='summary_metric'

rule = alt.Chart().mark_rule(color='red').encode(
    x='mean(value):Q'
)
# chart_two = alt.Chart().mark_rule().encode(
#     x='mean')

(chart_one + rule).facet(row='summary_metric', data=summary_melt)

alt.FacetChart(...)

In [219]:
roc_frame.head()

,bootstrap,fpr,tpr,threshold
0,0,0.0,0.000000,1.999981
1,0,0.0,0.022222,0.999981
2,0,0.0,0.088889,0.999906
3,0,0.0,0.133333,0.999573
4,0,0.0,0.177778,0.999527


In [220]:
alt.Chart(roc_frame).mark_line().encode(
    x = 'fpr',
    y = 'tpr',
    color = 'bootstrap',
)

alt.Chart(...)